In [1]:
import os
os.add_dll_directory('C:\\Program Files\\IBM\\SQLLIB\\BIN')
# conectar a la base de datos IBM Db2
import ibm_db

In [2]:
from queries import get_academic_history, get_socioeconomico, tasa_reprobacion, dificultad_materia
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
conn = ibm_db.connect('DATABASE=' + os.getenv('DATABASE') + ';HOSTNAME=' + os.getenv('HOSTNAME') + ';PORT=' + os.getenv('PORT') + ';PROTOCOL=TCPIP;UID=' + os.getenv('USERNAME_DB') + ';PWD=' + os.getenv('PASSWORD_DB') + ';', '', '')
# conn = ibm_db.connect(os.getenv('DATABASE'), os.getenv('USERNAME_DB'), os.getenv('PASSWORD_DB')) # os.getenv('HOSTNAME'), os.getenv('PORT')

if conn:
    print("Conexión exitosa")
else:
    print("Error al conectar")

Conexión exitosa


#### tasa_reprobacion

In [5]:
tasa_reprobacion

"with materias as (select distinct codigomateria,mat.nombre materia\n\t\t\tfrom espol.tbl_materia mat\n\t\t\tinner join espol.tbl_materia_malla mm on mm.idmateria=mat.idmateria\n\t\t\tinner join espol.tbl_malla_curricular mc on mm.idmalla=mc.idmalla and mc.estado='A'\n\t\t\tinner join espol.tbl_programa_academico pg on pg.idprograma=mc.idprograma and pg.codcarrera!='027' and pg.codcarrera!='028' and pg.codcarrera!='TE')\n\n, registros as (select m.codigomateria,ha.estado_mat_tomada,count(*) total\n\t\t\tfrom espol.historia_anio ha \n\t\t\tinner join materias m on m.codigomateria=ha.cod_materia_acad\n\t\t\twhere estado_mat_tomada in ('AP','RP')  \n\t\t\t\tand int(anio)>=2020\n\t\t\t\tand ha.termino in ('1S','2S')\n\t\t\tgroup by m.codigomateria,ha.estado_mat_tomada)\n\t\t\t\n, valores_abs as (select\tr.codigomateria, coalesce(rp.total,0) reprobados, sum(r.total) registrados\n\t\t\tfrom registros r\t\t\t\n\t\t\tleft join registros rp on rp.codigomateria=r.codigomateria and rp.estado_mat_

In [6]:
stmt_select  = ibm_db.exec_immediate(conn, tasa_reprobacion)

In [7]:
# Fetch all rows
data_list = []
result = ibm_db.fetch_assoc(stmt_select)
while result:
    # print(result)
    data_list.append(result)
    result = ibm_db.fetch_assoc(stmt_select)

In [8]:
df_tasa_reprobacion = pd.DataFrame(data_list)

In [9]:
df_tasa_reprobacion.shape

(741, 5)

In [10]:
df_tasa_reprobacion["CODIGOMATERIA"] = df_tasa_reprobacion["CODIGOMATERIA"].str.strip()

In [11]:
df_tasa_reprobacion.head()

,CODIGOMATERIA,MATERIA,REPROBADOS,REGISTRADOS,TASA_RP
0,DING2037,PORTAFOLIO,4,74,"5,41"
1,ACUG1035,ACUICULTURA ORNAMENTAL,1,104,"0,96"
2,ACUG1036,ANÁLISIS DE DATOS ACUÍCOLAS,2,91,"2,20"
3,ACUG1039,CULTIVO DE ESPECIES NO TRADICIONALES,1,80,"1,25"
4,ACUG1040,CULTIVO DE PLANCTON,1,68,"1,47"


In [12]:
df_tasa_reprobacion.to_csv("./data/materias/tasa_reprobacion.csv", index=False)

#### dificultad de materias

In [13]:
dificultad_materia

"with materias as (select distinct codigomateria,mat.nombre materia,clasifmateria\n\t\t\tfrom espol.tbl_materia mat\n\t\t\tinner join espol.tbl_materia_malla mm on mm.idmateria=mat.idmateria\n\t\t\tinner join espol.tbl_malla_curricular mc on mm.idmalla=mc.idmalla and mc.estado='A'\n\t\t\tinner join espol.tbl_programa_academico pg on pg.idprograma=mc.idprograma and pg.codcarrera!='027' and pg.codcarrera!='028' and pg.codcarrera!='TE')\n\nselect m.codigomateria,clasifmateria,materia,pa1.anio,pa1.termino,CAST(trunc(round (avg(promedio),2) ,2)  as decimal(5,2)) dificultad\n\t\t\tfrom espol.historia_anio ha \n\t\t\tinner join materias m on m.codigomateria=ha.cod_materia_acad \n\t\t\tinner join (select cod_materia_acad,min(idperiodo) idperiodo\n\t\t\t\t\t\tfrom espol.historia_anio ha\n\t\t\t\t\t\tinner join espol.tbl_periodo_academico pa on pa.anio=ha.anio and pa.termino=pa.termino and  pa.estado in ('E','A') and pa.tipo='R' and pa.tipoperiodo='S'\n\t\t\t\t\t\tgroup by cod_materia_acad) hd o

In [14]:
stmt_select  = ibm_db.exec_immediate(conn, dificultad_materia)

In [15]:
# Fetch all rows
data_list = []
result = ibm_db.fetch_assoc(stmt_select)
while result:
    # print(result)
    data_list.append(result)
    result = ibm_db.fetch_assoc(stmt_select)

In [16]:
df_dificultad_materia = pd.DataFrame(data_list)

In [17]:
df_dificultad_materia["CODIGOMATERIA"] = df_dificultad_materia["CODIGOMATERIA"].str.strip()

In [18]:
df_dificultad_materia.shape

(1926, 6)

In [19]:
df_dificultad_materia.head()

,CODIGOMATERIA,CLASIFMATERIA,MATERIA,ANIO,TERMINO,DIFICULTAD
0,ACUG1035,G,ACUICULTURA ORNAMENTAL,2020,0S,"8,52"
1,ACUG1036,G,ANÁLISIS DE DATOS ACUÍCOLAS,2020,0S,"7,72"
2,ACUG1039,G,CULTIVO DE ESPECIES NO TRADICIONALES,2020,0S,"8,63"
3,ACUG1040,G,CULTIVO DE PLANCTON,2020,0S,"8,25"
4,ACUG1041,G,ECOSISTEMAS ACUÍCOLAS,2020,0S,"7,41"


In [42]:
df_dificultad_materia["CLASIFMATERIA"].value_counts()

CLASIFMATERIA
G    1892
T      34
Name: count, dtype: int64

In [43]:
df_dificultad_materia[df_dificultad_materia["CLASIFMATERIA"] == "T"]

,CODIGOMATERIA,CLASIFMATERIA,MATERIA,ANIO,TERMINO,DIFICULTAD
20,ACUG1058,T,MATERIA INTEGRADORA DE ACUICULTURA,2020,0S,"9,47"
33,ADMG2031,T,MATERIA INTEGRADORA DE ADMINISTRACIÓN DE EMPRESAS,2020,0S,"9,22"
60,AGRG1029,T,MATERIA INTEGRADORA DE INGENIERÍA AGRÍCOLA Y B...,2020,0S,"9,60"
82,ALIG1040,T,MATERIA INTEGRADORA DE ALIMENTOS,2020,0S,"9,04"
95,ARQG2024,T,MATERIA INTEGRADORA EN ARQUEOLOGÍA,2021,0S,"8,16"
143,AUDG2040,T,MATERIA INTEGRADORA DE AUDITORÍA Y CONTROL DE ...,2020,0S,"9,53"
155,BIOG1027,T,MATERIA INTEGRADORA DE BIOLOGÍA,2020,0S,"9,26"
177,CADG2043,T,MATERIA INTEGRADORA DE PRODUCCIÓN PARA MEDIOS ...,2020,0S,"8,50"
200,CCPG1035,T,MATERIA INTEGRADORA DE COMPUTACIÓN,2020,0S,"8,90"
245,CIVG1054,T,MATERIA INTEGRADORA DE INGENIERÍA CIVIL,2020,0S,"8,76"


In [20]:
df_dificultad_materia.to_csv("./data/materias/dificultad_materia.csv", index=False)

In [21]:
df_dificultad_tasa = pd.merge(df_tasa_reprobacion, df_dificultad_materia, on="CODIGOMATERIA", how="left")

In [22]:
df_dificultad_tasa.to_csv("./data/materias/tasa_reprobacion_dificultad.csv", index=False)

#### Historia academica

In [23]:
cod_materia = "MATG1076"
year = "2025"
semester = "2S"

In [24]:
query = get_academic_history(cod_materia, year, semester)
query

"with materia_objetivo as (select hd.cod_estudiante,pa.idperiodo,hd.cod_materia_acad,hd.estado_mat_tomada,hd.vez_tomada,hd.nota1, hd.nota2, hd.promedio,hd.anio,hd.termino\n\t\tfrom espol.historia_anio hd\n\t\tinner join espol.tbl_periodo_academico pa on pa.anio=hd.anio and pa.termino=hd.termino and  pa.estado in ('E','A') and pa.tipo='R' and pa.tipoperiodo='S'\n\t\twhere hd.anio='2025' and hd.termino='2S'\n\t\tand hd.COD_MATERIA_ACAD = 'MATG1076' \n\t\tand estado_mat_tomada!='AN'\n\t\t)\n\n, v_beta as (select m.codigomateria,CAST(trunc(round (avg(ha.promedio),2) ,2)  as decimal(5,2)) nivel_beta\n\t\tfrom espol.historia_anio ha \n\t\tinner join (select distinct mat.codigomateria \n\t\t\tfrom espol.tbl_materia mat\n\t\t\tinner join espol.tbl_materia_malla mm on mm.idmateria=mat.idmateria\n\t\t\tinner join espol.tbl_malla_curricular mc on mm.idmalla=mc.idmalla and mc.estado='A'\n\t\t\tinner join espol.tbl_programa_academico pg on pg.idprograma=mc.idprograma and pg.codcarrera!='027' and pg

In [25]:
stmt_select  = ibm_db.exec_immediate(conn, query)

In [26]:
# Fetch all rows
data_list = []
result = ibm_db.fetch_assoc(stmt_select)
while result:
    result["COD_MATERIA_ACAD_MO"] = result["COD_MATERIA_ACAD_MO"].strip()
    # print(result)
    data_list.append(result)
    result = ibm_db.fetch_assoc(stmt_select)

In [27]:
df = pd.DataFrame(data_list)

In [28]:
df.shape

(10, 25)

In [29]:
df.head()

,COD_ESTUDIANTE,COD_MATERIA_ACAD_MO,ESTADO_MAT_TOMADA_MO,VEZ_TOMADA_MO,NOTA1_MO,NOTA2MO,PROMEDIO_MO,DIFICULTAD_MO,T_MAT_TOMADAS,PROM_1PARCIAL,PROM_2PARCIAL,PROM_CALIFICACIONES,MAT_APROBADAS,PROM_CALIF_APROBADAS,TERMINOS_REGISTRADOS,PERDIO_CARRERA,PROM_MAT_REPROBADAS1,PROM_MAT_REPROBADAS2,PROM_MAT_REPROBADAS3,MUY_FACIL,FACIL,MODERADA,DIFICIL,MUY_DIFICIL,GPA_ULTIMO
0,202206702,MATG1076,AC,1,0,0,"0,00","5,97",28,58,55,"6,34",20,"6,81",8,N,"5,18","5,05",None,0,0,0,0,3,"6,81"
1,202315784,MATG1076,AC,1,0,0,"0,00","5,97",18,66,67,"7,08",15,"7,49",5,N,"5,04",None,None,0,0,0,0,5,"7,40"
2,202200440,MATG1076,AC,1,0,0,"0,00","5,97",30,56,54,"6,33",21,"7,12",8,N,"4,86","3,27",None,0,0,0,0,3,"7,12"
3,202308359,MATG1076,AC,1,0,0,"0,00","5,97",23,87,78,"8,55",23,"8,55",6,N,None,None,None,0,0,0,1,3,"8,44"
4,202205811,MATG1076,AC,2,0,0,"0,00","5,97",30,60,47,"6,08",20,"6,86",8,N,"4,52",None,None,0,0,0,0,4,"6,86"


#### Socioeconomica

In [30]:
query = get_socioeconomico(df["COD_ESTUDIANTE"].to_list())
query

"SELECT vef.CODESTUDIANTE, vef.APELLIDOS, vef.NOMBRES, vef.EMAIL, vef.ANIO_TERMINO_INGRESO, vef.CATEGORIA, vef.ISE, vef.CARRERA, vef.UNIDAD,\n\t\tvef.TIENEDISCAPACIDAD, vef.SEXO, vef.AUTOIDENTIFICACIONGENERO, vef.AUTOIDENTIFICACIONETNICA, vef.ESTADOCIVIL, vef.FECHANACIMIENTO, vef.PAISNACIMIENTO,\n\t\tvef.PROVINCIANACIMIENTO, vef.CIUDADNACIMIENTO, vef.COLEGIO, vef.PAISCOLEGIO, vef.PROVINCIACOLEGIO, vef.CANTONCOLEGIO, vef.TIPOCOLEGIO, \n\t\tvef.ANIOGRADUACION, vef.CATEGORIACOLEGIO, vef.JORNADA, vef.BECACOLEGIO, vef.OTROSIDIOMAS, vef.COMIDASALDIA, vef.CONSUMODIARIOESPOL, \n\t\tvef.TIEMPOPROMEDIOLLEGARESPOL, vef.VECESBUSENTRADA, vef.VECESCARROENTRADA, vef.BICICLETAENTRADA, vef.VECESTAXIENTRADA, vef.VECESCARROMOTOAMIGOENTRADA,\n\t\tvef.VECESTRICIMOTOENTRADA, vef.CAMINAENTRADA, vef.VECESBUSSALIDA, vef.VECESCARROSALIDA, vef.BICICLETASALIDA, vef.VECESTAXISALIDA, vef.VECESCARROMOTOAMIGOSALIDA, \n\t\tvef.VECESTRICIMOTOSALIDA, vef.CAMINASALIDA, vef.NIVELINGLES, vef.POSEETARJETACREDITO, vef.POSEET

In [31]:
stmt_select  = ibm_db.exec_immediate(conn, query)

In [32]:
# Fetch all rows
data_list = []
result = ibm_db.fetch_assoc(stmt_select)
while result:
    # print(result)
    data_list.append(result)
    result = ibm_db.fetch_assoc(stmt_select)

In [33]:
df_socio = pd.DataFrame(data_list)

In [34]:
df_socio.shape

(10, 75)

In [35]:
df_socio.head()

,CODESTUDIANTE,APELLIDOS,NOMBRES,EMAIL,ANIO_TERMINO_INGRESO,CATEGORIA,ISE,CARRERA,UNIDAD,TIENEDISCAPACIDAD,SEXO,AUTOIDENTIFICACIONGENERO,AUTOIDENTIFICACIONETNICA,ESTADOCIVIL,FECHANACIMIENTO,PAISNACIMIENTO,PROVINCIANACIMIENTO,CIUDADNACIMIENTO,COLEGIO,PAISCOLEGIO,PROVINCIACOLEGIO,CANTONCOLEGIO,TIPOCOLEGIO,ANIOGRADUACION,CATEGORIACOLEGIO,JORNADA,BECACOLEGIO,OTROSIDIOMAS,COMIDASALDIA,CONSUMODIARIOESPOL,TIEMPOPROMEDIOLLEGARESPOL,VECESBUSENTRADA,VECESCARROENTRADA,BICICLETAENTRADA,VECESTAXIENTRADA,VECESCARROMOTOAMIGOENTRADA,VECESTRICIMOTOENTRADA,CAMINAENTRADA,VECESBUSSALIDA,VECESCARROSALIDA,BICICLETASALIDA,VECESTAXISALIDA,VECESCARROMOTOAMIGOSALIDA,VECESTRICIMOTOSALIDA,CAMINASALIDA,NIVELINGLES,POSEETARJETACREDITO,POSEETARJETADEBITO,ALIMENTACION,TRANSPORTE,NIVELINSTRUCCIONPADRE,NIVELINSTRUCCIONMADRE,ESTADOCIVILPADRES,DISCAPACIDAD,FAMILIARDISCAPACIDAD,ENFERMEDAD,FAMILIARENFERMEDAD,RECIBEBONO,DIFICULTADAPRENDIZAJE,PAISVIVE,PROVINCIAVIVE,CIUDADVIVE,TIPOVIVIENDASEP,CANTIDADCUARTOS,CANTIDADBANIO,METERIALTECHOVIVIENDASEP,METERIALPAREDVIVIENDASEP,VIAACCESOVIVIENDASEP,ABASTECIMIENTOAGUA,RECIBEAYUDA,VALORAYUDA,TIPOBACHILLER,COMIDAS,DISPOSITIVOS,ACCESOINTERNET
0,202200440,MENOSCAL CHOEZ,MARTIN JOSE,mjmenosc@espol.edu.ec,2022 1S,1,"0,2643",Matemática,Facultad de Ciencias Naturales y Matemáticas,N,Masculino,masculino,Mestizo,soltero,2002-03-19,ECUADOR,GUAYAS,GUAYAQUIL,REPLICA VICENTE ROCAFUERTE,ECUADOR,GUAYAS,GUAYAQUIL,Fiscal,0,8,None,Ninguna,NO,3,5,16 a 30 minutos,2,0,0,0,0,0,0,2,0,0,0,0,0,0,BÁSICO,NO,NO,280,50,None,None,Unión de Hecho,NO,None,NO,None,NO,None,ECUADOR,GUAYAS,GUAYAQUIL,Casa/Villa,3,1,Zinc /teja /eternit,Hormigón / ladrillo / bloque / cemento,Carretera / calle pavimentada o adoquinada / c...,RED PÚBLICA LAS 24 HORAS,SI,"100,00",None,DESAYUNO;ALMUERZO;MERIENDA,PORTÁTIL EXCLUSIVA;CELULAR,WIFI EXCLUSIVO DE LA VIVIENDA
1,202205811,CARRILLO AUQUILLA,MARIO ABEL,macarril@espol.edu.ec,2022 1S,1,"0,2542",Matemática,Facultad de Ciencias Naturales y Matemáticas,N,Masculino,masculino,Indigena,soltero,2002-08-19,ECUADOR,GUAYAS,GUAYAQUIL,LUIS BONINI PINO,ECUADOR,GUAYAS,GUAYAQUIL,Fiscal,2020,8,VESPERTINA (12:00-18:00 APROX.),Excelencia académica,NO,3,1,31 a 60 minutos,2,0,0,0,0,0,1,2,0,0,0,0,0,1,None,NO,NO,400,380,None,None,Casados,NO,None,NO,None,NO,None,ECUADOR,GUAYAS,GUAYAQUIL,Casa/Villa,2,2,Hormigón/losa/cemento,Hormigón / ladrillo / bloque / cemento,Carretera / calle pavimentada o adoquinada / c...,RED PÚBLICA LAS 24 HORAS,NO,"0,00",Bachillerato en Ciencias,DESAYUNO;ALMUERZO;CENA,CELULAR,WIFI EXCLUSIVO DE LA VIVIENDA;CYBER;RED PÚBLIC...
2,202206702,VELEZ ORDOÑEZ,ARIEL ALEXANDER,arialvel@espol.edu.ec,2022 1S,2,"0,4566",Matemática,Facultad de Ciencias Naturales y Matemáticas,N,Masculino,masculino,Mestizo,soltero,2002-09-09,ECUADOR,GUAYAS,GUAYAQUIL,CLAIRE BUCARAM DE AIVAS,ECUADOR,GUAYAS,GUAYAQUIL,Particular,2021,10,MATUTINA (07:00-14:00 APROX.),Ninguna,SI,3,0,61 a 90 minutos,1,0,0,0,0,0,0,1,0,0,0,0,0,0,BÁSICO,NO,SI,200,20,None,None,Divorciados,NO,None,NO,None,NO,None,ECUADOR,GUAYAS,GUAYAQUIL,Casa/Villa,2,1,Zinc /teja /eternit,Hormigón / ladrillo / bloque / cemento,Carretera / calle pavimentada o adoquinada / c...,RED PÚBLICA LAS 24 HORAS,NO,"0,00",Bachillerato en Ciencias,DESAYUNO;ALMUERZO;CENA,PC ESCRITORIO COMPARTIDA,WIFI COMPARTIDO
3,202305058,ALVAREZ PERALTA,FRANCISCO JOSE,fralpera@espol.edu.ec,2023 1S,2,"0,5579",Matemática,Facultad de Ciencias Naturales y Matemáticas,N,Masculino,masculino,Mestizo,soltero,2002-06-19,ECUADOR,GUAYAS,GUAYAQUIL,UNIDAD EDUCATIVA O´NEIL,ECUADOR,GUAYAS,GUAYAQUIL,Particular,2020,10,MATUTINA (07:00-14:00 APROX.),Ninguna,SI,3,3,31 a 60 minutos,2,0,0,0,0,0,0,2,0,0,0,0,0,0,INTERMEDIO,NO,NO,450,60,None,None,Casados,NO,None,NO,None,NO,None,ECUADOR,GUAYAS,GUAYAQUIL,Casa/Villa,3,1,Zinc /teja /eternit,Hormigón / ladrillo / bloque / cemento,Carretera / calle pavimentada o adoquinada / c...,RED PÚBLICA LAS 24 HORAS,NO,"0,00",Bachillerato en Ciencias,DESAYUNO;ALMUERZO;CENA,CELULAR,WIFI EXCLUSIVO DE LA VIVIENDA
4,202305249,ROBAL

In [36]:
df["COD_ESTUDIANTE"] = df["COD_ESTUDIANTE"].astype(str).str.strip()
df_socio["CODESTUDIANTE"] = df_socio["CODESTUDIANTE"].astype(str).str.strip()

In [37]:
df_merge = pd.merge(df, df_socio, left_on='COD_ESTUDIANTE', right_on="CODESTUDIANTE", how='left')

In [38]:
df_merge.shape

(10, 100)

In [39]:
df_merge.head()

,COD_ESTUDIANTE,COD_MATERIA_ACAD_MO,ESTADO_MAT_TOMADA_MO,VEZ_TOMADA_MO,NOTA1_MO,NOTA2MO,PROMEDIO_MO,DIFICULTAD_MO,T_MAT_TOMADAS,PROM_1PARCIAL,PROM_2PARCIAL,PROM_CALIFICACIONES,MAT_APROBADAS,PROM_CALIF_APROBADAS,TERMINOS_REGISTRADOS,PERDIO_CARRERA,PROM_MAT_REPROBADAS1,PROM_MAT_REPROBADAS2,PROM_MAT_REPROBADAS3,MUY_FACIL,FACIL,MODERADA,DIFICIL,MUY_DIFICIL,GPA_ULTIMO,CODESTUDIANTE,APELLIDOS,NOMBRES,EMAIL,ANIO_TERMINO_INGRESO,CATEGORIA,ISE,CARRERA,UNIDAD,TIENEDISCAPACIDAD,SEXO,AUTOIDENTIFICACIONGENERO,AUTOIDENTIFICACIONETNICA,ESTADOCIVIL,FECHANACIMIENTO,PAISNACIMIENTO,PROVINCIANACIMIENTO,CIUDADNACIMIENTO,COLEGIO,PAISCOLEGIO,PROVINCIACOLEGIO,CANTONCOLEGIO,TIPOCOLEGIO,ANIOGRADUACION,CATEGORIACOLEGIO,JORNADA,BECACOLEGIO,OTROSIDIOMAS,COMIDASALDIA,CONSUMODIARIOESPOL,TIEMPOPROMEDIOLLEGARESPOL,VECESBUSENTRADA,VECESCARROENTRADA,BICICLETAENTRADA,VECESTAXIENTRADA,VECESCARROMOTOAMIGOENTRADA,VECESTRICIMOTOENTRADA,CAMINAENTRADA,VECESBUSSALIDA,VECESCARROSALIDA,BICICLETASALIDA,VECESTAXISALIDA,VECESCARROMOTOAMIGOSALIDA,VECESTRICIMOTOSALIDA,CAMINASALIDA,NIVELINGLES,POSEETARJETACREDITO,POSEETARJETADEBITO,ALIMENTACION,TRANSPORTE,NIVELINSTRUCCIONPADRE,NIVELINSTRUCCIONMADRE,ESTADOCIVILPADRES,DISCAPACIDAD,FAMILIARDISCAPACIDAD,ENFERMEDAD,FAMILIARENFERMEDAD,RECIBEBONO,DIFICULTADAPRENDIZAJE,PAISVIVE,PROVINCIAVIVE,CIUDADVIVE,TIPOVIVIENDASEP,CANTIDADCUARTOS,CANTIDADBANIO,METERIALTECHOVIVIENDASEP,METERIALPAREDVIVIENDASEP,VIAACCESOVIVIENDASEP,ABASTECIMIENTOAGUA,RECIBEAYUDA,VALORAYUDA,TIPOBACHILLER,COMIDAS,DISPOSITIVOS,ACCESOINTERNET
0,202206702,MATG1076,AC,1,0,0,"0,00","5,97",28,58,55,"6,34",20,"6,81",8,N,"5,18","5,05",None,0,0,0,0,3,"6,81",202206702,VELEZ ORDOÑEZ,ARIEL ALEXANDER,arialvel@espol.edu.ec,2022 1S,2,"0,4566",Matemática,Facultad de Ciencias Naturales y Matemáticas,N,Masculino,masculino,Mestizo,soltero,2002-09-09,ECUADOR,GUAYAS,GUAYAQUIL,CLAIRE BUCARAM DE AIVAS,ECUADOR,GUAYAS,GUAYAQUIL,Particular,2021,10,MATUTINA (07:00-14:00 APROX.),Ninguna,SI,3,0,61 a 90 minutos,1,0,0,0,0,0,0,1,0,0,0,0,0,0,BÁSICO,NO,SI,200,20,None,None,Divorciados,NO,None,NO,None,NO,None,ECUADOR,GUAYAS,GUAYAQUIL,Casa/Villa,2,1,Zinc /teja /eternit,Hormigón / ladrillo / bloque / cemento,Carretera / calle pavimentada o adoquinada / c...,RED PÚBLICA LAS 24 HORAS,NO,"0,00",Bachillerato en Ciencias,DESAYUNO;ALMUERZO;CENA,PC ESCRITORIO COMPARTIDA,WIFI COMPARTIDO
1,202315784,MATG1076,AC,1,0,0,"0,00","5,97",18,66,67,"7,08",15,"7,49",5,N,"5,04",None,None,0,0,0,0,5,"7,40",202315784,MEJIA BARRERA,JOEL ANDRES,joanmeji@espol.edu.ec,2023 2S,2,"0,3649",Matemática,Facultad de Ciencias Naturales y Matemáticas,N,Masculino,masculino,Mestizo,soltero,2005-08-08,ECUADOR,GUAYAS,GUAYAQUIL,UNIDAD EDUCATIVA PARTICULAR SAN JOSEMARIA ESCRIVA,ECUADOR,GUAYAS,GUAYAQUIL,Particular,2023,10,VESPERTINA (12:00-18:00 APROX.),Ninguna,SI,3,5,31 a 60 minutos,2,0,0,0,0,0,1,1,0,0,0,0,0,1,BÁSICO,NO,NO,300,30,None,None,Viudo/a,NO,None,NO,None,NO,None,ECUADOR,GUAYAS,GUAYAQUIL,Casa/Villa,3,1,Zinc /teja /eternit,Hormigón / ladrillo / bloque / cemento,Carretera / calle pavimentada o adoquinada / c...,RED PÚBLICA LAS 24 HORAS,NO,"0,00",Bachillerato en Ciencias,DESAYUNO;ALMUERZO;MERIENDA,CELULAR,WIFI EXCLUSIVO DE LA VIVIENDA
2,202200440,MATG1076,AC,1,0,0,"0,00","5,97",30,56,54,"6,33",21,"7,12",8,N,"4,86","3,27",None,0,0,0,0,3,"7,12",202200440,MENOSCAL CHOEZ,MARTIN JOSE,mjmenosc@espol.edu.ec,2022 1S,1,"0,2643",Matemática,Facultad de Ciencias Naturales y Matemáticas,N,Masculino,masculino,Mestizo,soltero,2002-03-19,ECUADOR,GUAYAS,GUAYAQUIL,REPLICA VICENTE ROCAFUERTE,ECUADOR,GUAYAS,GUAYAQUIL,Fiscal,0,8,None,Ninguna,NO,3,5,16 a 30 minutos,2,0,0,0,0,0,0,2,0,0,0,0,0,0,BÁSICO,NO,NO,280,50,None,None,Unión de Hecho,NO,None,NO,None,NO,None,ECUADOR,GUAYAS,GUAYAQUIL,Casa/Villa,3,1,Zinc /teja /eternit,Hormigón / ladrillo / bloque / cemento,Carretera / calle pavimentada o adoquinada / c...,RED PÚBLICA LAS 24 HORAS,SI,"100,00",None,DESAYUNO;ALMUERZO;MERIENDA,PORTÁTIL EXCLUSIVA;CELULAR,WIFI EXCLUSIVO DE LA VIVIENDA
3,202308359,MATG1076,AC,1,0,0,"0,

In [40]:
df_merge.to_csv("./data/materias/"+ cod_materia + "_" + year + "_" + semester + ".csv", index=False)

### Cerrar coneccion

In [41]:
# Close the connection
ibm_db.close(conn)

True